### Logistic Regression on Household Income Level and Energy Consumption Behavior.

In [282]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [352]:
octo = pd.read_csv("Power-Networks-LCL-June2015(withAcornGps)v2_25.csv")

In [347]:
feb = pd.read_csv("Power-Networks-LCL-June2015(withAcornGps)v2_24.csv")

In [354]:
octo.head()

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
0,MAC000882,Std,2012-07-14 06:00:00.0000000,0.106,ACORN-H,Comfortable
1,MAC000882,Std,2012-07-14 06:30:00.0000000,0.074,ACORN-H,Comfortable
2,MAC000882,Std,2012-07-14 07:00:00.0000000,0.128,ACORN-H,Comfortable
3,MAC000882,Std,2012-07-14 07:30:00.0000000,0.159,ACORN-H,Comfortable
4,MAC000882,Std,2012-07-14 08:00:00.0000000,0.104,ACORN-H,Comfortable


In [355]:
feb.tail()

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
999995,MAC000882,Std,2012-07-14 03:30:00.0000000,0.068,ACORN-H,Comfortable
999996,MAC000882,Std,2012-07-14 04:00:00.0000000,0.068,ACORN-H,Comfortable
999997,MAC000882,Std,2012-07-14 04:30:00.0000000,0.057,ACORN-H,Comfortable
999998,MAC000882,Std,2012-07-14 05:00:00.0000000,0.065,ACORN-H,Comfortable
999999,MAC000882,Std,2012-07-14 05:30:00.0000000,0.197,ACORN-H,Comfortable


In [318]:
octo.DateTime = pd.to_datetime(octo.DateTime)
feb.DateTime = pd.to_datetime(feb.DateTime)

In [320]:
octo = octo[369:]
octo.head()

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
738,MAC000002,Std,2012-10-27 09:30:00,0.389,ACORN-A,Affluent
739,MAC000002,Std,2012-10-27 10:00:00,0.856,ACORN-A,Affluent
740,MAC000002,Std,2012-10-27 10:30:00,0.299,ACORN-A,Affluent
741,MAC000002,Std,2012-10-27 11:00:00,0.221,ACORN-A,Affluent
742,MAC000002,Std,2012-10-27 11:30:00,0.18,ACORN-A,Affluent


## Numerize the economic category

In [321]:
dic = {'Affluent': 1 , 'Adversity': 2, 'Comfortable': 3, 'ACORN-U': 4}

In [322]:
octo.Acorn_grouped = octo.Acorn_grouped.map(dic)
feb.Acorn_grouped = feb.Acorn_grouped.map(dic)

//anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [323]:
label_octo = octo.iloc[:, (0, -1)].drop_duplicates()
label_octo.head()

,LCLid,Acorn_grouped
738,MAC000002,1
24158,MAC000003,2
59627,MAC000004,1
91304,MAC000006,2
127765,MAC000007,3


In [324]:
label_feb = feb.iloc[:, (0, -1)].drop_duplicates()
label_feb.head()

,LCLid,Acorn_grouped
0,MAC000840,1
23838,MAC000842,2
56147,MAC000843,2
88404,MAC000844,3
120603,MAC000846,3


In [325]:
octo = octo[octo.DateTime.dt.month == 10] # cut from 10-22 to 10:31

In [326]:
feb = feb[feb.DateTime.dt.month == 10] # cut from 10-22 to 10:31

In [327]:
octo.head()

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
738,MAC000002,Std,2012-10-27 09:30:00,0.389,ACORN-A,1
739,MAC000002,Std,2012-10-27 10:00:00,0.856,ACORN-A,1
740,MAC000002,Std,2012-10-27 10:30:00,0.299,ACORN-A,1
741,MAC000002,Std,2012-10-27 11:00:00,0.221,ACORN-A,1
742,MAC000002,Std,2012-10-27 11:30:00,0.18,ACORN-A,1


In [328]:
feb.head()

,LCLid,stdorToU,DateTime,KWH/hh (per half hour),Acorn,Acorn_grouped
0,MAC000840,Std,2012-10-19 17:30:00,0.379,ACORN-E,1
1,MAC000840,Std,2012-10-19 18:00:00,0.22,ACORN-E,1
2,MAC000840,Std,2012-10-19 18:30:00,0.188,ACORN-E,1
3,MAC000840,Std,2012-10-19 19:00:00,0.176,ACORN-E,1
4,MAC000840,Std,2012-10-19 19:30:00,1.115,ACORN-E,1


In [329]:
octo.iloc[:,3] = octo.iloc[:,3].astype(float)
feb.iloc[:,3] = feb.iloc[:,3].astype(float)

In [330]:
octo_p = octo.pivot_table(values='KWH/hh (per half hour) ', index = "DateTime", columns = "LCLid" )
feb_p = feb.pivot_table(values='KWH/hh (per half hour) ', index = "DateTime", columns = "LCLid" )

In [331]:
#octo_p = octo_p.dropna(axis = 0, how = "any")
#feb_p = feb_p.dropna(axis = 0, how = "any")
feb_p = feb_p.T
octo_p = octo_p.T

In [332]:
octo_p = octo_p.reset_index()
feb_p = feb_p.reset_index()

In [333]:
octo_p = octo_p.merge(label_octo, on = "LCLid", how = 'inner')
feb_p = feb_p.merge(label_feb, on = "LCLid", how = 'inner')

In [303]:
octo_mean = np.mean(octo_p.iloc[:, 1:-1], axis = 1)
octo_std = np.std(octo_p.iloc[:, 1:-1], axis = 1) 

In [334]:
df = feb_p.ix[:, 131:]
df = pd.concat([feb_p.LCLid, df], axis = 1)

In [338]:
df.iloc[:, -459:]

,2013-10-22 11:00:00,2013-10-22 11:30:00,2013-10-22 12:00:00,2013-10-22 12:30:00,2013-10-22 13:00:00,2013-10-22 13:30:00,2013-10-22 14:00:00,2013-10-22 14:30:00,2013-10-22 15:00:00,2013-10-22 15:30:00,...,2013-10-31 19:30:00,2013-10-31 20:00:00,2013-10-31 20:30:00,2013-10-31 21:00:00,2013-10-31 21:30:00,2013-10-31 22:00:00,2013-10-31 22:30:00,2013-10-31 23:00:00,2013-10-31 23:30:00,Acorn_grouped
0,0.375,0.666,0.222,0.105,0.211,0.101,0.095,0.154,0.667,0.263,...,0.513,0.179,0.189,0.160,0.194,1.054,0.573,0.530,0.220,1
1,0.574,0.689,0.601,0.565,0.504,0.086,0.072,0.261,0.072,0.093,...,0.578,0.567,0.547,0.583,0.588,0.321,0.092,0.086,0.106,2
2,0.164,0.225,0.217,0.040,0.081,0.085,0.158,0.107,0.184,0.144,...,0.244,0.169,0.181,0.050,0.052,0.005,0.041,0.005,0.043,2
3,0.283,0.113,0.098,0.061,0.114,0.212,0.161,0.330,0.139,0.176,...,0.282,0.455,0.359,0.437,0.413,0.460,0.598,0.339,0.451,3
4,0.022,0.039,0.015,0.041,0.019,0.022,0.043,0.067,0.090,0.100,...,0.253,0.122,0.166,0.130,0.134,0.058,0.040,0.071,0.042,3
5,0.063,0.079,0.064,0.062,0.222,1.061,0.114,0.076,0.081,0.036,...,0.497,0.556,0.335,0.059,0.056,0.055,0.047,0.057,0.052,1
6,0.028,0.072,0.103,0.105,0.083,0.094,0.098,0.193,0.097,0.449,...,0.257,0.112,0.241,0.117,0.144,0.129,0.111,0.026,0.003,1
7,0.100,0.145,0.071,0.075,0.114,0.061,0.136,0.174,0.106,0.157,...,0.527,0.516,0.520,0.550,0.482,0.339,0.449,1.150,0.426,1
8,0.042,0.029,0.054,0.020,0.062,0.007,0.082,0.016,0.051,0.027,...,0.103,0.131,0.095,0.121,0.120,0.103,0.122,0.110,0.133,2
9,0.092,0.089,0.180,0.095,0.137,0.185,0.118,0.131,0.344,0.191,...,0.313,0.124,0.116,0.130,0.131,0.117,0.118,0.118,0.109,2


In [339]:
feb_mean = np.mean(df.iloc[:, :-1], axis = 1)
feb_std = np.std(df.iloc[:, :-1], axis = 1) 

In [340]:
octo_p.iloc[:, 1:-1] = octo_p.iloc[:, 1:-1].apply(lambda x: (x - octo_mean)/octo_std)
df.iloc[:, 1:-1] = df.iloc[:, 1:-1].apply(lambda x: (x - feb_mean)/feb_std)

In [341]:
df_nor = df.copy()

In [343]:
df_nor.head()

,LCLid,2012-10-03 17:00:00,2012-10-03 17:30:00,2012-10-03 18:00:00,2012-10-03 18:30:00,2012-10-03 19:00:00,2012-10-03 19:30:00,2012-10-03 20:00:00,2012-10-03 20:30:00,2012-10-03 21:00:00,...,2013-10-31 19:30:00,2013-10-31 20:00:00,2013-10-31 20:30:00,2013-10-31 21:00:00,2013-10-31 21:30:00,2013-10-31 22:00:00,2013-10-31 22:30:00,2013-10-31 23:00:00,2013-10-31 23:30:00,Acorn_grouped
0,MAC000840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.755460,-0.388242,-0.353999,-0.453302,-0.336878,2.607981,0.960915,0.813672,-0.247847,1
1,MAC000842,-0.364322,-0.164411,-0.334335,-0.199395,-0.339333,-0.244375,-0.139422,-0.369320,-0.324340,...,2.134569,2.079594,1.979638,2.159558,2.184547,0.850139,-0.294353,-0.324340,-0.224384,2
2,MAC000843,-0.213812,1.290274,-0.357058,0.737753,1.392593,1.791636,0.226159,0.052217,-0.776565,...,1.546071,0.778680,0.901463,-0.438913,-0.418450,-0.899348,-0.531000,-0.899348,-0.510537,2
3,MAC000844,-0.726391,-0.502257,-0.634533,-0.502257,-0.193613,0.442047,0.802132,-0.031942,0.092985,...,0.081962,0.717622,0.364886,0.651484,0.563300,0.735994,1.243052,0.291399,0.702925,3
4,MAC000846,0.035886,0.017801,0.379502,2.658215,2.001126,1.301838,1.440490,0.831627,0.246878,...,0.909996,0.120283,0.385530,0.168510,0.192623,-0.265531,-0.374041,-0.187162,-0.361984,3


In [344]:
octo_p.head()

DateTime,LCLid,2012-10-01 00:00:00,2012-10-01 00:30:00,2012-10-01 01:00:00,2012-10-01 01:30:00,2012-10-01 02:00:00,2012-10-01 02:30:00,2012-10-01 03:00:00,2012-10-01 03:30:00,2012-10-01 04:00:00,...,2013-10-31 19:30:00,2013-10-31 20:00:00,2013-10-31 20:30:00,2013-10-31 21:00:00,2013-10-31 21:30:00,2013-10-31 22:00:00,2013-10-31 22:30:00,2013-10-31 23:00:00,2013-10-31 23:30:00,Acorn_grouped
0,MAC000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.353526,1.493429,-0.222068,-0.293547,-0.293547,-0.218306,-0.357502,-0.237117,-0.410171,1
1,MAC000003,-0.659557,2.000452,1.897093,0.093202,-0.593252,0.147806,-0.622504,-0.201271,-0.491844,...,-0.595202,-0.604953,-0.626405,-0.620554,-0.655657,-0.612753,-0.269527,-0.585451,-0.663457,2
2,MAC000004,-0.490749,-0.490749,2.035244,-0.490749,-0.490749,-0.490749,-0.490749,-0.490749,1.785421,...,-0.490749,-0.490749,2.215672,-0.490749,-0.490749,-0.490749,-0.490749,1.868695,-0.490749,1
3,MAC000006,-0.764997,-0.593333,-1.051105,-0.478890,-0.898514,-1.012957,-0.345374,-1.032031,-0.745924,...,0.112398,-0.040193,-0.326300,0.379431,-0.326300,-0.097414,0.264988,-0.230931,0.322210,2
4,MAC000007,-0.616594,-0.548367,-0.716659,-0.566561,-0.734852,-0.698465,-0.602948,-0.534722,-0.584755,...,0.079314,0.415896,0.515961,0.129346,-0.020752,0.547800,0.202121,0.102056,-0.466496,3


In [236]:
octo_p.Acorn_grouped = octo_p.Acorn_grouped.astype(str)

## Let's run the Logistic Regression

In [254]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(C=1e6)
logit.fit(np.c_[octo_p.iloc[:,1:]], octo_p.Acorn_grouped)
logit.intercept_

array([-0.0059231 ,  0.00424578, -0.01642361, -0.01982414])

In [258]:
predlabels = logit.predict(np.c_[octo_p.iloc[:,1:]])
predlabels = predlabels.astype(str)

In [259]:
truelabels = octo_p.Acorn_grouped

In [260]:
from sklearn.metrics import confusion_matrix
confusion_matrix(truelabels, predlabels)

array([[13,  0,  0,  0],
       [ 0, 11,  0,  0],
       [ 0,  0,  5,  0],
       [ 0,  0,  0,  1]])